In [1]:
from data_admin import Database
Data_admin = Database()

In [4]:
import json
import requests
import time
import tqdm

def sentiment_scores(text):
    token = 'xxx'
    url = 'https://aip.baidubce.com/rpc/2.0/nlp/v1/sentiment_classify?charset=UTF-8&access_token={}'.format(token)

    data = {
        'text':str(text)
    }
    data = json.dumps(data)
    res = requests.post(url,data=data)
    json_data = json.loads(res.text)
    try:
        positive_prob = json_data['items'][0]['positive_prob']
        sentiment = json_data['items'][0]['sentiment']
        return positive_prob, sentiment
    except:
        return 0,0
text = '#疫情#忽然感觉这次疫情空前紧张，都开始建方舱医院了'
print(sentiment_scores(text))
all_score = []

(0.31834, 0)


In [3]:
sql_1 = 'select answer_text from zhihu_car'
data_1 = Data_admin.database(sql_1)
import pandas as pd
data = pd.DataFrame(data_1)
data.columns =['text']
data

,text
0,执行力，才是拉开人与人差距的关键\n人生一大悲剧，便是只懂得在心里谋局布阵生活中却瞻前顾后，...
1,先来看看问题的本质：想不到和自己朝夕相处一起逃课一起打游戏一起喝醉的同学，居然比自己发展的好...
2,本来差距就很大，只是学校生活就那么点事儿，不显而已。\n\n上学的时候，我们只能通过分数衡量...
3,产生现象的原因\n\n不少老师都说过同一句话，都是同一个教室同样的老师教出来的学生，差别怎么...
4,谢邀，这个问题的问法，让大家都觉得毕业好像才是起跑线，\n但事实是，学校毕业其实是某种意义上...
...,...
35883,说因为开便利店轻松才选择开便利店的人，基本可以不看好，你最好还是别做。便利店是表面看劳动强度...
35884,高锤子高，知乎这群卖发财经的真是要脸。\n租金，水电，人工，损耗，退货。这几座大山现在是越来...
35885,便利店就是个幸苦钱除非你开连锁店当资本家剥削店员，不然就是一最底层劳动人民的活。\n挣不挣钱...
35886,10％左右的利润空间，地理位置很重要奥。放弃20万的工作去干超市我是没见过了。


In [15]:
for index in tqdm.tqdm(data.index[25989:]):
    text = data.loc[index].values[0]
    positive_prob,sentiment = sentiment_scores(text[0:600])
    all_score.append([index+1,text,positive_prob,sentiment])
    time.sleep(0.3)

100%|██████████| 9899/9899 [1:37:18<00:00,  1.70it/s]  


In [16]:
all = pd.DataFrame(all_score)
all.columns = ['id','text','positive_prob','sentiment']
all

,id,text,positive_prob,sentiment
0,1,执行力，才是拉开人与人差距的关键\n人生一大悲剧，便是只懂得在心里谋局布阵生活中却瞻前顾后，...,0.012754,0
1,2,先来看看问题的本质：想不到和自己朝夕相处一起逃课一起打游戏一起喝醉的同学，居然比自己发展的好...,0.075941,0
2,3,本来差距就很大，只是学校生活就那么点事儿，不显而已。\n\n上学的时候，我们只能通过分数衡量...,0.824485,2
3,4,产生现象的原因\n\n不少老师都说过同一句话，都是同一个教室同样的老师教出来的学生，差别怎么...,0.342615,0
4,5,谢邀，这个问题的问法，让大家都觉得毕业好像才是起跑线，\n但事实是，学校毕业其实是某种意义上...,0.557738,2
...,...,...,...,...
35883,35884,说因为开便利店轻松才选择开便利店的人，基本可以不看好，你最好还是别做。便利店是表面看劳动强度...,0.011631,0
35884,35885,高锤子高，知乎这群卖发财经的真是要脸。\n租金，水电，人工，损耗，退货。这几座大山现在是越来...,0.000020,0
35885,35886,便利店就是个幸苦钱除非你开连锁店当资本家剥削店员，不然就是一最底层劳动人民的活。\n挣不挣钱...,0.002468,0
35886,35887,10％左右的利润空间，地理位置很重要奥。放弃20万的工作去干超市我是没见过了。,0.040787,0


In [17]:
my_data = all[['id','positive_prob','sentiment']]
# print(my_data,my_data.loc[1][1])

for index in tqdm.tqdm(my_data.index):
    sql_2 = "update zhihu_car set positive_prob={} , sentiment = {} where answer_num={}".format(my_data.loc[index].values[1], int(my_data.loc[index].values[2]), my_data.loc[index].values[0])
    Data_admin.database(sql_2)

100%|██████████| 35888/35888 [00:38<00:00, 929.18it/s] 


In [18]:
data2 = all.loc[all['positive_prob']==0]
data2.index = range(len(data2))
data2

,id,text,positive_prob,sentiment
0,10,,0.0,0
1,39,,0.0,0
2,229,https://www.zhihu.com/question/33971854最近，近二十年...,0.0,0
3,241,,0.0,0
4,318,,0.0,0
...,...,...,...,...
365,34582,,0.0,0
366,34591,,0.0,0
367,34598,,0.0,0
368,34600,,0.0,0


In [19]:
import warnings

warnings.filterwarnings("ignore")
for index in tqdm.tqdm(data2.index):
    text = data2.loc[index].values[1]
    positive_prob,sentiment = sentiment_scores(text[0:600])
    data2['positive_prob'][index] = positive_prob
    data2['sentiment'][index] = sentiment
    time.sleep(0.5)

100%|██████████| 370/370 [04:27<00:00,  1.38it/s]


In [20]:
data2

,id,text,positive_prob,sentiment
0,10,,0.000000,0
1,39,,0.000000,0
2,229,https://www.zhihu.com/question/33971854最近，近二十年...,0.944941,2
3,241,,0.000000,0
4,318,,0.000000,0
...,...,...,...,...
365,34582,,0.000000,0
366,34591,,0.000000,0
367,34598,,0.000000,0
368,34600,,0.000000,0


In [21]:
my_data2 = data2[['id','positive_prob','sentiment']]

for index in tqdm.tqdm(my_data2.index):
    sql_3 = "update zhihu_car set positive_prob={} , sentiment = {} where answer_num={}".format(my_data2.loc[index].values[1], int(my_data2.loc[index].values[2]), my_data2.loc[index].values[0])
    Data_admin.database(sql_3)

100%|██████████| 370/370 [00:00<00:00, 888.19it/s]
